# Inversion Prep

Here we provide a general template for setting up a tomographic inversion using ObsPy + Pyatoa. This will involve agathering an event catalog with moment tensors, collecting observation waveforms and response files, organizing data into the optimal directory structure, and generating ASDFDataSets that can be used in a SeisFlows or standalone inversion.


## Alaska Event Catalog

Alaska is a good region for an example problem, let's work there. First we'll use ObsPy to gather our initial catalog of events from the past decade in a box bounding Anchorage and Fairbanks.

In [1]:
from obspy import UTCDateTime, Catalog
from obspy.clients.fdsn import Client

In [2]:
c = Client("USGS")
cat = c.get_events(starttime=UTCDateTime("2010-01-01T00:00:00"), 
                   endtime=UTCDateTime("2020-01-01T00:00:00"), 
                   maxdepth=60.0,
                   minmagnitude=5.0,
                   maxmagnitude=6.0, 
                   minlatitude=59.75, 
                   maxlatitude=65.50, 
                   minlongitude=-154.5, 
                   maxlongitude=-143.789
                  )
cat

15 Event(s) in Catalog:
2019-01-13T16:45:55.437000Z | +61.299, -150.065 | 5.0 ml | manual
2019-01-06T03:45:34.525000Z | +65.407, -153.280 | 5.1 ml | manual
...
2011-06-16T19:06:05.214000Z | +60.765, -151.076 | 5.1 mw | manual
2011-01-23T02:50:04.629000Z | +63.542, -150.865 | 5.2 mw | manual
To see all events call 'print(CatalogObject.__str__(print_all=True))'

Lets have a look at the Event IDs of our events. If we knew these apriori, we could have gathered our catalog based on event ids

In [3]:
from pyatoa.utils.form import format_event_name

event_ids = []
for event in cat:
    event_ids.append(format_event_name(event))
event_ids

['ak019lrs7iu',
 'ak0199za3yf',
 'ak0191pccr7',
 'ak018fe5jk85',
 'ak20421672',
 'ak018fcpk9xi',
 'us1000hyge',
 'ak018fcntv5m',
 'ak018dsf3btv',
 'ak017f7s3c06',
 'ak014dlss56k',
 'ak014b5xf1in',
 'ak013ae2ycca',
 'ak0117oi3hnt',
 'ak011122ukq6']

## Getting moment tensors

Great, we have an event catalog now, but to do waveform simulations we need moment tensors. 

Unfortunately it's not straight forward to grab moment tensor information directly from USGS as they do not directly provide XML files. It would be possible to manually generate moment tensor objects from each [individual event pages](https://earthquake.usgs.gov/earthquakes/eventpage/ak019lrs7iu/moment-tensor), but that seems tedious for a tutorial. 

Instead we'll use [GCMT](https://www.globalcmt.org/CMTsearch.html). Pyatoa has a function to read .ndk files hosted online with GCMT, finding events based on origintime and magnitude.

In [4]:
from pyatoa.core.gatherer import get_gcmt_moment_tensor

events = []
for event in cat:
    origintime = event.preferred_origin().time
    magnitude = event.preferred_magnitude().mag
    try:
        events.append(get_gcmt_moment_tensor(origintime, magnitude))
    except FileNotFoundError:
        print(f"No GCMT event found for: {format_event_name(event)}")
        continue
    
gcmt_catalog = Catalog(events)
print(f"\n{len(gcmt_catalog)}/{len(cat)} events with GCMT solutions found")

No GCMT event found for: ak018fcpk9xi
No GCMT event found for: us1000hyge
No GCMT event found for: ak018fcntv5m
No GCMT event found for: ak013ae2ycca

11/15 events with GCMT solutions found


Great, 11 out of 15 isn't bad, we'll go ahead with and use the GCMT catalog that we just collected. However if we wanted to retain the (probably more accurate) origin information from the USGS catalog, we would need to move the moment tensor objects from the GCMT catalog over to the USGS catalog, an exercise left for the reader...

## Gathering Observation Data

Now we need seismic waveform data for all the events in our catalog. We can use the multithreaded data gathering functioality of Pyatoa's Gatherer class. First we need to determine the available broadband stations in the area, using ObsPy. 

Some pieces of relevant information that help motivate our search:
*  The Alaska Earthquake Center (AEC) operates stations under the network code "AK".
*  The SEED standard seismometer instrument code is "H"
*  The SEED standard for broadband instruments is "B" or "H"


In [5]:
c = Client("IRIS")
inv = c.get_stations(network="AK", 
                     station="*", 
                     location="*",
                     channel="BH?",
                     starttime=UTCDateTime("2010-01-01T00:00:00"), 
                     endtime=UTCDateTime("2020-01-01T00:00:00"), 
                     minlatitude=59.75,                    
                     maxlatitude=65.50, 
                     minlongitude=-154.5, 
                     maxlongitude=-143.789,
                     level="channel"
                    )
inv

Inventory created at 2022-03-02T19:37:12.579000Z
	Created by: IRIS WEB SERVICE: fdsnws-station | version: 1.1.48
		    http://service.iris.edu/fdsnws/station/1/query?starttime=2010-01-01...
	Sending institution: IRIS-DMC (IRIS-DMC)
	Contains:
		Networks (1):
			AK
		Stations (76):
			AK.BMR (Bremner River, AK, USA)
			AK.BPAW (Bear Paw Mountain, AK, USA)
			AK.BRLK (Bradley Lake, AK, USA)
			AK.BWN (Browne, AK, USA)
			AK.CAPN (Captain Cook Nikiski, AK, USA)
			AK.CAST (Castle Rocks, AK, USA)
			AK.CCB (Clear Creek Butte, AK, USA)
			AK.CHUM (Lake Minchumina, AK, USA)
			AK.CUT (Chulitna, AK, USA)
			AK.DDM (Donnely Dome, AK, USA)
			AK.DHY (Denali Highway, AK, USA)
			AK.DIV (Divide Microwave, AK, USA)
			AK.DOT (Dot Lake, AK, USA)
			AK.EYAK (Cordova Ski Area, AK, USA)
			AK.FIB (Fire Island, AK, USA)
			AK.FID (Fidalgo, AK, USA)
			AK.FIRE (Fire Island, AK, USA)
			AK.GHO (Gloryhole, AK, USA)
			AK.GLB (Gilahina Butte, AK, USA)
			AK.GLI (Glacier Island, AK, USA)
			AK.GLM (Gilmore 

In [6]:
# We'll need to create a list of station ids for data gathering
station_codes = []
for net in inv:
    for sta in net:
        station_codes.append(f"{net.code}.{sta.code}.*.BH?")
        
# Let's just take a look at the first 10 as an example
station_codes[:10]

['AK.BMR.*.BH?',
 'AK.BPAW.*.BH?',
 'AK.BRLK.*.BH?',
 'AK.BWN.*.BH?',
 'AK.CAPN.*.BH?',
 'AK.CAST.*.BH?',
 'AK.CCB.*.BH?',
 'AK.CHUM.*.BH?',
 'AK.CUT.*.BH?',
 'AK.DDM.*.BH?']

If we look at the inventory we see that there are 76 available stations in our domain, quite a lot! Lets see how many have waveform data for the events in question. We will do this by creating an ASDFDataSet for a single event, and trying to fill it with all available data.

In [7]:
from pyasdf import ASDFDataSet
from pyatoa import Gatherer, Config

In [8]:
# Here we are just using the first event in our catalog
event = gcmt_catalog[0]
event_id = format_event_name(event)

In [9]:
# The gatherer needs to know where to look (Client) and when to look (origintime)
cfg = Config(client="IRIS")
origintime = event.preferred_origin().time

In [10]:
# Now we initate the Gatherer and use its multithreading capabilities to gather waveform and metadata
# Here the 'return_count' argument means we only want to save stations that return data including 
# metadata (1) + 3 waveforms (3) = 4 

# First make sure were writing to an empty dataset
ds_fid = f"../tests/test_data/docs_data/{event_id}.h5"
if os.path.exists(ds_fid):
    os.remove(ds_fid)
    
with ASDFDataSet(ds_fid) as ds:
    ds.add_quakeml(event)
    gthr = Gatherer(config=cfg, ds=ds, origintime=origintime)
    gthr.gather_obs_multithread(codes=station_codes, return_count=4, print_exception=True)

NameError: name 'os' is not defined

In [11]:
with ASDFDataSet(f"../tests/test_data/docs_data/{event_id}.h5") as ds:
    print(ds.waveforms.list())
    print(f"\n{len(ds.waveforms.list())} stations collected")

['AK.BMR', 'AK.BPAW', 'AK.BRLK', 'AK.BWN', 'AK.CAPN', 'AK.CAST', 'AK.CCB', 'AK.CHUM', 'AK.CUT', 'AK.DIV', 'AK.DOT', 'AK.EYAK', 'AK.FIRE', 'AK.GHO', 'AK.GLB', 'AK.GOAT', 'AK.HDA', 'AK.HIN', 'AK.HMT', 'AK.KAI', 'AK.KLU', 'AK.KNK', 'AK.KTH', 'AK.MCK', 'AK.NEA2', 'AK.NICH', 'AK.PAX', 'AK.PPLA', 'AK.PWL', 'AK.RAG', 'AK.RC01', 'AK.RIDG', 'AK.RND', 'AK.SAW', 'AK.SCM', 'AK.SCRK', 'AK.SKN', 'AK.SLK', 'AK.SWD', 'AK.TRF', 'AK.WRH']

41 stations collected


Great! Looks like we've got data for 41 stations just for this one event. Some stations did not return any data, as expected, but many of them returned a StationXML plus three component waveforms (as explained by data_count == 4).

___ 
### Next Steps

Now you can repeat the above data gathering steps for the remainder of the events in your catalog. Each event should get it's own ASDFDataSet to keep data organized nicely. Take a look at the Storage tutorial to get an idea of how to navigate and manipulate the ASDFDataSets. Also have a look at the Pyaflowa tutorial in order to figure out how to process the data you've just collected, either in a standalone manner using Pyatao + SPECFEM3D, or with an automated workflow tool like SeisFlows.